# Devoir 1 - Vérification de code

### A) Simplifier et établir le problème stationnaire: 

##### a. Préciser de quel type est le problème résultant en stationnaire (c’est-à-dire parabolique, hyperbolique ou elliptique, mixte)

**Réponse :** 
Équation de base:

$$\frac{\partial C}{\partial t} = D_{eff} \nabla^2 C - S$$

Pour un problème stationnaire, l'équation ne dépends pas sur le temps donc $\frac{\partial C}{\partial t} = 0$

Notre équation deviens donc: 

$$ 0= D_{eff} \nabla^2 C - S \iff S = D_{eff} \nabla^2 C$$

##### b. Préciser votre choix de système de coordonnées

**Réponse :** 
    
On utilise un système de coordonnées *cylindrique*: $$C(r,\theta, z)$$



##### c. Réduire au maximum la dimensionnalité du problème (c-à-d appliquer les simplifications de circonstances), préciser l’existence ou non de symétrie dans le domaine et justifier vos choix

**Réponse :** 

**Simplifications**

1- Hauteur infinie $\rightarrow$ le probleme ne depends pas de la hauteur $z$

2- Symetrie radiale $\rightarrow$ le probleme ne depends pas de l'angle $\theta$

Notre probleme est donc en $1D$, et ne depends que du rayon $r$: 

$$C(r)$$



##### d. Présenter en conséquence une discrétisation du domaine en Ntot=5 nœuds (faire un schéma et montrer la position exacte de tous les nœuds sur le rayon). Préciser la taille des intervalles Δr choisis

**Réponse :** 

Rayon $R = \frac{D}2 = 0.5 m$

Noeuds $N = 5$

donc $Δr = \frac{0.5}5 = 0.125m$

Ci-dessus se trouve le schéma de la discrétisation:
    

![Devoir_1\images\schema_discret.png](attachment:image.png)


##### e. D’après l’énoncé, préciser:
* les conditions frontières (et leurs types) nécessaires à la résolution du problème;

* la condition initiale requise, si nécessaire

**Réponse :** 
    
2 conditions fontières sont nécéssaires pour la résolution du problème: 

1- À $r = 0.5$: Condition de Drichlet, avec $$\left. C \right|_{r=0.5} = C_e$$

2-  À $r = 0$: Condition de Neumann, avec $$\left. \frac{\partial C}{\partial r} \right|_{r=0} = 0$$

En régime stationnaire, le problème ne requiert pas de conditions initiales.


### B) Résoudre analytiquement en considérant les conditions frontières choisies et obtenir le profil de concentration à l’état stationnaire :

##### a. Spécifiquement, montrer que le profil de concentration en sel est de forme quadratique et s’écrit comme:

$C(r) = \frac{1}{4}\,\frac{S}{D_{\mathrm{eff}}}\,R^{2}
\left(\frac{r^{2}}{R^{2}} - 1\right) + C_{e}$


**Réponse :** 
    
    Mettre réponse ici


### C) En donnant suffisamment de détails et vous basant sur les diapos du rappel concernant lesdifférences finies (cf. Moodle - diapo 4), approximer l’Éq. (1) à l’état stationnaire, avec un termesource constant, et les conditions frontières appropriées par les schémas de différenciation suivants en espace :

$\left.\frac{\partial C}{\partial r}\right|_{i}
= \frac{C_{i+1}-C_i}{\Delta r},
\qquad
\left.\frac{\partial^2 C}{\partial r^2}\right|_{i}
= \frac{C_{i+1}-2C_i+C_{i-1}}{\Delta r^2}$

##### a. Préciser l’équation obtenue en chacun des nœuds (incluant les nœuds frontières)

**Réponse :** 
    
    Mettre réponse ici

##### b. Préciser la procédure générale pour résoudre le problème

**Réponse :** 
    
    Mettre réponse ici


##### c. Préciser l’erreur de troncature et donc l’ordre de précision attendu du schéma global

**Réponse :** 
    
    Mettre réponse ici


### D) Écrire un code de calcul générique (c-à-d pour un Ntot non précisé d’avance) pour résoudre l’Éq. (1) (à l’état stationnaire et avec S constant), puis :

##### a. Tracer le profil de concentration obtenu à l’état stationnaire et comparer le à la solution analytique (Eq.(2)) - préciser clairement tous les paramètres de la simulation utilisés

**Réponse :** 
    


##### b. Faire une vérification du code appropriée comme vu en classe, en utilisant la solution analytique (Eq.(2)). Entre autres, mais pas seulement, tracer sur un même graphique les erreurs L1, L2 et L∞

**Réponse :** 
    
    Mettre réponse ici

### E) Remplacer maintenant les schémas de différenciation précédents par ceux-ci et les implanter dans votre code :

$\left.\frac{\partial C}{\partial r}\right|_{i}
= \frac{C_{i+1}-C_{i-1}}{2\,\Delta r},
\qquad
\left.\frac{\partial^2 C}{\partial r^2}\right|_{i}
= \frac{C_{i+1}-2C_i+C_{i-1}}{\Delta r^2}$



##### a. Préciser l’erreur de troncature et donc l’ordre de précision attendu de ce nouveau schéma global

**Réponse :** 
    
    Mettre réponse ici

##### b. Refaire les vérifications

**Réponse :** 
    
    Mettre réponse ici

##### c. Tracer les profils de concentration obtenus avec les deux schémas numériques (questions C/D et E) et comparer à la solution analytique (question B)

**Réponse :** 
    
    Mettre réponse ici

##### d. Que constatez-vous maintenant ? Comment expliquer le résultat obtenu ?

**Réponse :** 
    
    Mettre réponse ici